# CPA Device Flow (Client Mode)

In this step we show how to obtain an access token for use with a service provider, but **without** associating the client with a user account.

## Setup

Execute the following line to load the libraries:


In [ ]:
require('/opt/notebooks/tutorial/lib/tutorial-utils')(this);

## Making an unauthenticated request

Initially, the client does not have an access token, so will receive a `401 Unauthorized` error when making a request to the service provider. The following request uses the RadioTAG protocol.

![Unauthenticated request](/notebooks/tutorial/images/uml/unauthenticated-request.png)

In [ ]:
this.post("http://sp:8002/radiodns/tag/1/tag", {
  bearer: "0.c221.ce15.ce1.dab",
  time: 1441792800,
  time_source: "user"
}, { form: true });

## WWW-Authenticate header

The `401 Unauthorized` response contains a `WWW-Authenticate` header indicating that CPA authentication is supported by this service provider. The header contains several values:

* `version`: The protocol version in use, currently 1.0.
* `name`: The name of the authorization provider.
* `uri`: The base URL of the authorization provider.
* `modes`: A comma-separated list of available authorization modes.

## Client mode

![Client mode](/notebooks/tutorial/images/uml/client-mode.png)

## Registering the client

To register with the authorization parameter, the client makes a `POST /register` request with the following parameters:

* `client_name`: human-readable name of the client.
* `software_id`: an identifier for the client software
    application.
* `software_version`: a version identifier for the client software application.

The URL is constructed by appending `/register` to the `uri` value from the `WWW-Authenticate` header.

In [ ]:
this.post("http://ap:8001/register", {
  client_name: "My test client",
  software_id: "cpa-test-client",
  software_version: "1.0"
});

## Obtaining an access token

To obtain an access token, the client makes `POST /token` requests with the following parameters:

* `grant_type`: The value `"http://tech.ebu.ch/cpa/1.0/client_credentials"`.
* `client_id`: The unique identifier for the client given by the authorization provider.
* `client_secret`: A shared secret value between the client and authorization provider.
* `domain`: The service provider domain.

In [ ]:
this.post("http://ap:8001/token", {
  grant_type: "http://tech.ebu.ch/cpa/1.0/client_credentials",
  client_id: "<client_id>",
  client_secret: "<client_secret>",
  domain: "sp:8002"
});

## Using the access token

To use the access token in a request to the service provider, using the RadioTAG protocol, we include the token in an `Authorization` header.

![Authenticated request](/notebooks/tutorial/images/uml/authenticated-request.png)

In [ ]:
this.post("http://sp:8002/radiodns/tag/1/tag", {
  time_source: "user",
  bearer: "0.c221.ce15.ce1.dab",
  time: 1441792800
}, { form: true, token: "<access_token>" });

## Viewing the tags

You can see the tags stored by the Service Provider [here](http://localhost:8002/tags/all).

## Next

Go to [Refreshing an Expired Token](05_CPA_Tutorial_Token_Refresh.ipynb).